## After multiple observations and including the $\epsilon$ parameter

Below is  function that: 
* takes in a $\theta$, a vector of $\beta$s, $\epsilon$ and a few other numbers
* generates a data sequence from the model by sampling 




In [1]:
import random
import pandas as pd
import numpy as np

In [2]:
import mcmcs_guess as mg

In [3]:
epsilons = np.linspace(0,0.5,num=11)
muts = np.linspace(-1,1,num=41)

In [4]:
# choose arbitrary initial value of theta (chosen from a normal distribution)
theta = np.random.normal()

# number of problems to be solved
len_beta = 20

# choose arbitrary vector of betas of length len_beta
betas = np.random.randn(len_beta,)

# means and standard deviations
sigt,mub,sigb = 1., 0., 1.

# number of steps/samples: should make 10000 and discount the first 1000 for burn in reasons
nsteps = 10000 

g=0.2

In [5]:
# possible scores:
# scores = range(len_beta)
scores = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

In [6]:
# BE CAREFUL about running this! Will take HOURS!! prints out how long each step takes, the corresponding params and the theta estimates 

import timeit
 
preds = [] 

for mut in muts: # loop through mu thetas
    for eps in epsilons: # loop through epsilons
        
        sim_list = [] 
        
        start = timeit.default_timer()

        for num in scores: # loop through each score
            sim = mg.MCMC(theta,betas,mut,sigt,mub,sigb,nsteps,len_beta,eps,num) # do mcmc at that particular score
            
            theta_scores = [] 
            for th in sim['thetas']: # take all the thetas and move them into score space
                theta_scores.append(mg.sigmoid_guess(th,g)*len_beta)

            estimate = np.mean(theta_scores[1000:]) # take the mean for this batch after the first 1000 iterations
            
            sim_list.append(estimate)
        
        stop = timeit.default_timer()
        print('time: ', stop - start)
        
        print('epsilon: ', eps)
        print('mut: ', mut)
        print(sim_list)
        
        preds.append(sim_list)
    

time:  405.79422087967396
epsilon:  0.0
mut:  -1.0
[4.436886190429306, 4.532883376923237, 4.583250870713071, 4.620517341994319, 4.825134740934322, 4.900059648499076, 5.111080849995281, 5.3226805817685126, 5.514067500705239, 5.845579777305127, 6.571125064459952, 6.8269649604974605, 7.91542783580589, 8.670761995594004, 9.859382805319633, 10.77062890303425, 11.541496904442791, 12.647231470869572, 13.281115245416569, 14.358880282200035, 15.429699962745472]
time:  424.36162876337767
epsilon:  0.05
mut:  -1.0
[4.4691756191091, 4.5392378499773915, 4.563940280762544, 4.680191569602658, 4.729546293886822, 4.909001154057888, 4.984712624139633, 5.140023963582886, 5.446868165282027, 5.835913070915723, 6.236715773369158, 6.937193408240636, 7.477120833492502, 8.049186730832364, 9.216378600727843, 10.17903522746154, 11.250742652897928, 11.938365694300883, 13.29269878602749, 13.88320950371549, 14.801926791025364]
time:  435.06244741380215
epsilon:  0.1
mut:  -1.0
[4.511379666091572, 4.577638725533939,

In [7]:
len(preds) # check that the right number came out

451

In [8]:
# transfer predictions to a pandas dataframe for model comparisons and save to CSV for future reference
preds_basic = pd.DataFrame(preds)
preds_basic = preds_basic.transpose() # make each column a prediction
preds_basic.to_csv('Guess/preds451_guess_1.csv', index=False, header=True)
preds_basic

0          1          2          3          4          5    \
0    4.436886   4.469176   4.511380   4.543780   4.542982   4.576877   
1    4.532883   4.539238   4.577639   4.586846   4.547330   4.577433   
2    4.583251   4.563940   4.586205   4.599785   4.627784   4.651980   
3    4.620517   4.680192   4.623535   4.699736   4.695977   4.734594   
4    4.825135   4.729546   4.683464   4.762634   4.736310   4.725548   
5    4.900060   4.909001   4.775638   4.801169   4.798459   4.852183   
6    5.111081   4.984713   4.884977   4.922833   4.868719   4.827119   
7    5.322681   5.140024   5.015314   4.947018   5.009849   4.932763   
8    5.514068   5.446868   5.114991   5.205161   5.080488   5.150638   
9    5.845580   5.835913   5.647672   5.417016   5.226288   5.283870   
10   6.571125   6.236716   5.844584   5.728643   5.664741   5.438679   
11   6.826965   6.937193   6.419839   6.031026   5.671300   5.461375   
12   7.915428   7.477121   6.843361   6.472631   6.413316   5.779643   
13   8.670762   8.049187   7.455894   6.961357   6.364654   6.094855   
14   9.859383   9.216379   8.729008   7.850930   7.203857   6.495736   
15  10.770629  10.179035   9.689170   8.595162   8.244238   7.060480   
16  11.541497  11.250743  10.526834  10.012603   8.690331   7.563781   
17  12.647231  11.938366  11.659645  10.672360   9.665741   8.279220   
18  13.281115  13.292699  12.730316  11.570058  10.548132   9.201214   
19  14.358880  13.883210  13.477359  12.592973  11.813802  10.128488   
20  15.429700  14.801927  14.227511  13.523501  12.356978  11.220891   

         6         7         8         9    ...        441        442  \
0   4.653449  4.704717  4.767320  4.904434  ...   8.069431   8.130490   
1   4.654070  4.702108  4.837789  4.960528  ...   8.377763   8.719339   
2   4.702160  4.781979  4.795197  4.935211  ...   8.886301   9.315479   
3   4.770259  4.726751  4.901007  5.027707  ...   9.453128   9.837046   
4   4.850540  4.792281  4.888346  4.951120  ...  10.092776  10.569970   
5   4.796258  4.879903  4.886592  4.994851  ...  10.809251  11.199075   
6   4.993224  4.977029  4.952842  4.955117  ...  11.458766  12.106035   
7   4.899698  4.981769  5.034324  5.038442  ...  12.147653  12.557104   
8   5.085887  5.084950  5.062906  5.078886  ...  12.770174  13.679414   
9   5.150200  5.045687  5.084559  5.053682  ...  13.565800  14.293207   
10  5.256656  5.272723  5.018193  5.119727  ...  14.459225  15.230593   
11  5.444947  5.287952  5.227813  5.188494  ...  15.148994  16.189230   
12  5.743987  5.443163  5.232742  5.201035  ...  15.875701  16.833694   
13  5.796526  5.426979  5.326827  5.215465  ...  16.639668  17.513966   
14  6.253574  5.778949  5.453785  5.205870  ...  17.340502  18.175868   
15  6.250211  5.919550  5.515830  5.248065  ...  17.986239  18.533804   
16  6.710774  6.032711  5.636507  5.199518  ...  18.400935  18.838380   
17  7.261096  6.402016  5.761458  5.252757  ...  18.951303  19.025506   
18  7.800817  6.512617  5.839618  5.278838  ...  19.115756  19.293797   
19  8.717446  6.815696  5.888160  5.484257  ...  19.312274  19.377018   
20  8.731842  7.418549  6.112502  5.530189  ...  19.468382  19.452664   

          443        444        445        446        447        448  \
0    9.130416   9.611589  11.403625  13.094625  16.042770  17.748258   
1    9.380760  10.333520  11.655715  14.072566  16.398868  17.752755   
2   10.057980  10.918386  12.706221  14.808628  16.813440  17.952879   
3   10.931681  11.474084  13.370552  15.890986  17.360273  18.071339   
4   11.309145  12.665503  14.452359  16.310678  17.590237  18.302265   
5   12.039812  13.488758  15.300578  16.470627  17.767268  18.408178   
6   13.165363  14.243932  16.014027  17.190837  18.058261  18.522008   
7   13.817026  15.160881  16.559238  17.645503  18.379789  18.621601   
8   14.687350  16.051723  17.207062  17.951704  18.487519  18.770959   
9   15.668433  16.774973  17.774992  18.331376  18.609773  18.801592   
10  16.453051  17.382490  